# Analyzing models

**cameo** uses and extends the model data structures defined by [cobrapy](https://opencobra.github.io/cobrapy/), our favorite **CO**nstraints-**B**ased **R**econstruction and **A**nalysis tool for **Py**thon. **cameo** is thus 100% compatible with **cobrapy**. For efficiency reasons though **cameo** implements its own analysis methods that take advantage of a more advanced solver interface.

In [1]:
from cameo import models
model = models.bigg.e_coli_core

## Flux Variability Analysis

Flux variability analysis (FVA) enables the computation of lower and upper bounds of reaction fluxes.

In [2]:
from cameo import flux_variability_analysis

In [3]:
fva_result = flux_variability_analysis(model)
fva_result.data_frame

,upper_bound,lower_bound
ACALD,0.00,-2.000000e+01
ACALDt,0.00,-2.000000e+01
ACKr,0.00,-2.000000e+01
ACONTa,20.00,-1.483300e-30
...,...,...
THD2,333.22,0.000000e+00
TKT1,20.00,-1.545362e-01
TKT2,20.00,-4.663728e-01
TPI,10.00,-1.000000e+01


In [4]:
fva_result.plot(index=fva_result.data_frame.index[:25])

One very useful application of FVA is determining if alternative optimal solution exist.

In [5]:
fva_result2 = flux_variability_analysis(model,fraction_of_optimum=0.5)
fva_result2.data_frame

,upper_bound,lower_bound
ACALD,0.000000e+00,-12.602453
ACALDt,4.733165e-30,-12.602453
ACKr,0.000000e+00,-13.358852
ACONTa,1.383029e+01,0.471437
...,...,...
THD2,1.686767e+02,0.000000
TKT1,1.328068e+01,-0.154536
TKT2,1.312294e+01,-0.466373
TPI,9.565355e+00,-3.793497


In [6]:
fva_result2.plot()

In [7]:
from cameo.visualization import plotting

Phenotpic Phase Plane
--------------------

The phenotypic phase plane is a modeling technique was developed to do a theoretical accessement of what cell can or cannot do in terms of the stoichiometric constraints [Edawards *et al.* 2001]. 

The phenotipic phase plane between growth and a product of interest yields the production envelope: a representation between the trade of between production of the desired product and growth.

In [8]:
from cameo import phenotypic_phase_plane

In [9]:
model.reactions.EX_o2_e.lower_bound = -10
result = phenotypic_phase_plane(model, 
                                variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                objective=model.reactions.EX_succ_e,
                                points=10)

In [10]:
result.plot(width=700)

The production envelope allows is a quick way to inspect the limitations of the system to design and how the production relates for with growth. In the previous example, succinate prudction is completly decoupled from growth and by decreasing the growth rate it is theoretically possible to produce up to 15 times more succinate. 

In [11]:
result.plot(height=350, points=[(0.52, 0), (0.23, 12.2)], points_colors=["green", "red"])

The production envelope can show the coupling between growth and production. There is no stoichiometric couple between growth and production for succinate under aerobic conditions, but that is not the case for acetate under anaerobic conditions.

In [12]:
result = phenotypic_phase_plane(model, 
                                variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                objective=model.reactions.EX_ac_e)
result.plot(height=350)

In [13]:
result.data_frame

,BIOMASS_Ecoli_core_w_GAM,objective_lower_bound,objective_upper_bound
0,0.559051,9.90568,11.503269
1,0.529627,2.62796,11.950466
2,0.500203,0.00000,12.397662
3,0.470779,0.00000,12.844858
...,...,...,...
16,0.088271,0.00000,18.658411
17,0.058847,0.00000,19.105607
18,0.029424,0.00000,19.552804
19,0.000000,0.00000,20.000000


One can imediatly see if a design is feasible within the new defined constraints.

In [14]:
result.plot(height=350, points=[(0.2, 8), (0.2, 2)], points_colors=["green", "red"])

The computed data can be inspected in the format of a pandas data frame by calling `result.data_frame`

In [15]:
result.data_frame

,BIOMASS_Ecoli_core_w_GAM,objective_lower_bound,objective_upper_bound
0,0.559051,9.90568,11.503269
1,0.529627,2.62796,11.950466
2,0.500203,0.00000,12.397662
3,0.470779,0.00000,12.844858
...,...,...,...
16,0.088271,0.00000,18.658411
17,0.058847,0.00000,19.105607
18,0.029424,0.00000,19.552804
19,0.000000,0.00000,20.000000


In [16]:
model.reactions.EX_o2_e.lower_bound = 0
result2 = phenotypic_phase_plane(model, 
                                 variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                 objective=model.reactions.EX_ac_e,
                                 points=10)
result2.plot()

## Flux Balance Impact Degree

In [17]:
from cameo.flux_analysis.analysis import flux_balance_impact_degree

In [18]:
model.reactions.EX_o2_e.lower_bound = -10

In [19]:
%time fbid = flux_balance_impact_degree(model, ["EX_o2_e"])

CPU times: user 340 ms, sys: 9.72 ms, total: 350 ms
Wall time: 452 ms


In [20]:
fbid

,perturbed
ACALD,False
ACALDt,False
ACKr,False
ACt2r,False
...,...
SUCCt3,False
SUCDi,False
THD2,False
TPI,False
